In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import os
import cv2
import numpy as np
import yaml
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.cuda.amp import autocast, GradScaler

In [2]:
# Трансформация. Преобразование данных в другой тип (исходно как PIL импортирует).
# Compose - трансформация, которая объединяет в себе другие трансформации. (массив трансформаций)

trans = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)), # если в датасете изображения разного размера
    tv.transforms.ToTensor(),         # Превращение PIL в тензор
    tv.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # тк хотим использовать модель vgg (а эти mean и std описаны на сайте pytorch)
])

dataset_path = "animals10"
dataset_train = tv.datasets.ImageFolder(
    root=dataset_path,
    transform=trans
)

In [3]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(
    dataset_train,
    shuffle=True,
    batch_size=batch_size,
    num_workers=0,
    drop_last=True
)

In [ ]:
# проверка что работает train_loader
for sample in train_loader:
    print(sample)
    
    break

In [10]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [6]:
model = tv.models.vgg19(weights=tv.models.vgg.VGG19_Weights)

/Users/egoryakovlev/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /Users/egoryakovlev/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [01:18<00:00, 7.30MB/s] 


In [12]:
model_resnet = tv.models.resnet50(weights=tv.models.ResNet50_Weights)

/Users/egoryakovlev/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/egoryakovlev/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:13<00:00, 7.62MB/s]


In [11]:
count_params(model)

143667240

In [15]:
count_params(model.features)

20024384

In [16]:
count_params(model.classifier)

123642856

In [18]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [13]:
count_params(model_resnet)

25557032

In [36]:
# Пишем свой классификатор

classifier = nn.Sequential(
    nn.Linear(25088, 100),
    nn.LeakyReLU(0.2),
    nn.Linear(100, 10)
)

# Применяем классификатор к модели
model.classifier = classifier

In [24]:
count_params(classifier)

2510920

In [25]:
# переобучаем только классификатор
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))

loss_fn = nn.CrossEntropyLoss()
sheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.6)

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
fn_loss = loss_fn.to(device)

In [31]:
def accuracy(pred, label):
    answer = F.softmax(pred.detach()).numpy().argmax(1) == label.numpy().argmax(1)
    return answer.mean()

In [29]:
use_amp = True
scaler = torch.cuda.amp.GradScaler() # только для GPU

/Users/egoryakovlev/opt/anaconda3/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [30]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

Эти строки кода используются для настройки поведения библиотеки cuDNN при работе с GPU в PyTorch, и их выбор может существенно влиять на производительность и воспроизводимость вычислений.

### 1. `torch.backends.cudnn.benchmark = True`
Эта строка включает режим benchmark в cuDNN. Когда `benchmark` установлен в `True`, PyTorch запускает несколько испытательных прогонов, чтобы определить оптимальные параметры для конкретного слоя (например, размер блока для сверток). Это может значительно ускорить обучение и инференс, особенно если у вас стабильные входные размеры.

**Преимущества:**
- **Ускорение вычислений:** Оптимизирует производительность путем выбора наиболее быстрого алгоритма для текущей конфигурации.

**Недостатки:**
- **Время на старт:** На начальном этапе потребуется время для проведения тестов, что может увеличить время первого запуска.
- **Подходит только для стабильных размеров входных данных:** Если размеры входных данных меняются, эта оптимизация может не сработать.

### 2. `torch.backends.cudnn.deterministic = False`
Эта строка отключает детерминированное поведение cuDNN. В случае, когда `deterministic` установлен в `False`, cuDNN может использовать не полностью детерминированные алгоритмы, которые могут приводить к различным результатам при каждом запуске.

**Преимущества:**
- **Ускорение вычислений:** Нехоть детерминированные алгоритмы могут быть быстрее и эффективнее по сравнению с их детерминированными аналогами.

**Недостатки:**
- **Невоспроизводимость:** Результаты могут немного различаться между запусками, что затрудняет отладку и воспроизводимость экспериментов.

### Когда использовать эти настройки?
- **Для производительности:** Используйте `torch.backends.cudnn.benchmark = True` и `torch.backends.cudnn.deterministic = False`, если вам важна максимальная производительность и стабильные размеры входных данных.
- **Для воспроизводимости:** Установите `torch.backends.cudnn.deterministic = True` (и, возможно, `torch.backends.cudnn.benchmark = False`), если вам необходимо, чтобы результаты были воспроизводимы при каждом запуске.

Эти настройки помогают балансировать между скоростью и воспроизводимостью, в зависимости от конкретных потребностей вашего проекта.

In [ ]:
epochs = 10
loss_epochs_list = []
acc_epochs_list = []
for epoch in range(epochs):
    loss_val = 0
    acc_val = 0
    for sample in (pbar := tqdm(train_loader)):
        img, label = sample
        label = F.one_hot(label, 10).float()
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        
        with autocast(use_amp):
            pred = model(img)
            loss = loss_fn(pred, label)

        scaler.scale(loss).backward()
        loss_item = loss.item()
        loss_val += loss_item

        scaler.step(optimizer)
        scaler.update()

        acc_current = accuracy(pred.cpu().float(), label.cpu().float())
        acc_val += acc_current

        pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')
    scheduler.step()
    loss_epochs_list += [loss_val/len(train_loader)]
    acc_epochs_list += [acc_val/len(train_loader)]
    print(loss_epochs_list[-1])
    print(acc_epochs_list[-1])

Процесс обучения нейронной сети с использованием PyTorch.Описание шагов:

1. **Параметры обучения**:
   - `epochs = 10`: Количество эпох обучения, то есть полных проходов по всему набору данных.
   - `loss_epochs_list = []`: Список для сохранения значений функции потерь после каждой эпохи.
   - `acc_epochs_list = []`: Список для сохранения значений точности после каждой эпохи.

2. **Цикл по эпохам**:
   - `for epoch in range(epochs)`: Цикл по числу эпох.
   - `loss_val = 0`: Переменная для накопления значения функции потерь за эпоху.
   - `acc_val = 0`: Переменная для накопления значения точности за эпоху.

3. **Цикл по батчам (пакетам) данных**:
   - `for sample in (pbar := tqdm(train_loader))`: Итерация по батчам данных из загрузчика `train_loader` с использованием прогресс-бара `tqdm`.

4. **Подготовка данных**:
   - `img, label = sample`: Получение изображений и меток из текущего батча.
   - `label = F.one_hot(label, 10).float()`: Преобразование меток в one-hot представление и преобразование к типу float.
   - `img = img.to(device)`: Перенос изображений на устройство (например, GPU).
   - `label = label.to(device)`: Перенос меток на устройство.

5. **Обнуление градиентов**:
   - `optimizer.zero_grad()`: Обнуление градиентов перед обратным проходом.

6. **Форвардный проход и вычисление функции потерь**:
   - `with autocast(use_amp)`: Контекстный менеджер для автоматического приведения типов (если используется mixed precision).
   - `pred = model(img)`: Прогон данных через модель для получения предсказаний.
   - `loss = loss_fn(pred, label)`: Вычисление функции потерь.

7. **Обратный проход и обновление весов**:
   - `scaler.scale(loss).backward()`: Масштабирование и обратный проход для вычисления градиентов.
   - `loss_item = loss.item()`: Получение значения функции потерь.
   - `loss_val += loss_item`: Добавление значения функции потерь к общему значению за эпоху.
   - `scaler.step(optimizer)`: Обновление весов модели.
   - `scaler.update()`: Обновление скейлера.

8. **Вычисление точности**:
   - `acc_current = accuracy(pred.cpu().float(), label.cpu().float())`: Вычисление точности для текущего батча.
   - `acc_val += acc_current`: Добавление значения точности к общему значению за эпоху.

9. **Обновление прогресс-бара**:
   - `pbar.set_description(f'loss: {loss_item:.5f}\taccuracy: {acc_current:.3f}')`: Обновление описания прогресс-бара значениями функции потерь и точности.

10. **Шаг планировщика обучения**:
    - `scheduler.step()`: Обновление параметров планировщика обучения.

11. **Сохранение значений функции потерь и точности за эпоху**:
    - `loss_epochs_list += [loss_val/len(train_loader)]`: Добавление среднего значения функции потерь за эпоху в список.
    - `acc_epochs_list += [acc_val/len(train_loader)]`: Добавление среднего значения точности за эпоху в список.
    - `print(loss_epochs_list[-1])`: Печать последнего значения функции потерь.
    - `print(acc_epochs_list[-1])`: Печать последнего значения точности.


In [ ]:
plt.title("Loss")
plt.plot(loss_epoch_list)

In [ ]:
plt.title("Accuracy")
plt.plot(acc_epochs_list)

In [43]:
img = sample[0][0:1]

In [ ]:
with torch.no_grad():
    ans = model(img.cuda())

In [ ]:
F.softmax(ans.cpu())